# Spark

In [2]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [3]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [4]:
f = distData.map(lambda x: x % 2 == 0)
f.take(3)

[False, True, False]

In [5]:
f = distData.filter(lambda x: x % 2 == 0)
f.take(5)

[2, 4]

# Hladanie prvocisel

In [6]:
# prevzane z https://districtdatalabs.silvrback.com/getting-started-with-spark-in-python

def isprime(n):
    """
    check if integer n is a prime
    """
    # make sure n is a positive integer
    n = abs(int(n))
    # 0 and 1 are not primes
    if n < 2:
        return False
    # 2 is the only even prime number
    if n == 2:
        return True
    # all other even numbers are not primes
    if not n & 1:
        return False
    # range starts with 3 and only needs to go up the square root of n
    # for all odd numbers
    for x in range(3, int(n**0.5)+1, 2):
        if n % x == 0:
            return False
    return True

In [7]:
import time

In [8]:
nums = sc.parallelize(range(10**6))

In [9]:
start = time.time()
print(nums.filter(isprime).count())
end = time.time()
print("Elapsed time: {} s".format(end - start))

78498
Elapsed time: 2.5926029682159424 s


### Nedistribuovany vypocet

In [10]:
l_nums = sc.parallelize(range(10**6), 1)

In [11]:
start = time.time()
print(l_nums.filter(isprime).count())
end = time.time()
print("Elapsed time: {} s".format(end - start))

78498
Elapsed time: 4.0168726444244385 s


### Delayed evaluation

In [12]:
start = time.time()
primes = nums.filter(isprime)
end = time.time()
print("Elapsed time: {} s".format(end - start))

Elapsed time: 9.036064147949219e-05 s


v predchadzajucej bunke sa este nic nevykonalo, nepouzil som funkiu, ktora by mala vracat nejaky vysledok, tak sa len pripravil vypocet na spustenie, ale nespustil sa

In [13]:
start = time.time()
# print nums.filter(isprime).take(5)
print(nums.filter(isprime).takeOrdered(5, key = lambda x: -x))
end = time.time()
print("Elapsed time: {} s".format(end - start))

[999983, 999979, 999961, 999959, 999953]
Elapsed time: 2.526235818862915 s


# MapReduce na spracovanie suboru

In [14]:
# Teraz tie data natahujem z jednueho suboru na disku jedneho fyzickeho pocitaca, ale mohol by som pouzit 
# natiahnutie z dristribuovaneho suoroveho systemu a vykonavat ten vypocet na celom clustri. 
# Jediny rozdiel by bol v tomto riadku
distFile = sc.textFile("data/shakespeare.txt")

In [15]:
distFile.count()

147928

In [16]:
distFile.first()

'Project Gutenberg’s The Complete Works of William Shakespeare, by William'

In [17]:
distFile.take(5)

['Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions']

In [18]:
tmp = distFile.filter(lambda line: "JULIA" in line)

In [19]:
tmp.count()

119

In [20]:
tmp.take(5)

['  JULIA, a lady of Verona, beloved of Proteus',
 "SCENE II. Verona. The garden Of JULIA'S house",
 'Enter JULIA and LUCETTA',
 '  JULIA. But say, Lucetta, now we are alone,',
 '  JULIA. Of all the fair resort of gentlemen']

In [21]:
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

5546150

In [22]:
distFile.filter(lambda line: "JULIA" in line).map(lambda s: len(s)).reduce(lambda a, b: a + b)

5408